# Fissio Base - Power Plant Data Explorer

This notebook connects to the seeded DuckDB database containing:
- **WRI Global Power Plant Database** (34,936 plants worldwide)
- **NRC Reactor Status** (last 365 days)
- **US Nuclear Plants** summary tables

In [ ]:
import duckdb

# Connect to the seeded database
con = duckdb.connect('/home/jovyan/data/fissio.duckdb', read_only=True)
print("Connected to fissio.duckdb")

## Available Tables

In [ ]:
con.sql("""
    SELECT table_schema, table_name, table_type
    FROM information_schema.tables
    WHERE table_schema IN ('plants', 'regulatory', 'market')
    ORDER BY table_schema, table_name
""").show()

## Global Power Plants by Fuel Type

In [ ]:
con.sql("""
    SELECT 
        primary_fuel,
        COUNT(*) as plant_count,
        ROUND(SUM(capacity_mw), 0) as total_mw,
        ROUND(AVG(capacity_mw), 1) as avg_mw
    FROM plants.global_power_plants
    GROUP BY primary_fuel
    ORDER BY total_mw DESC
""").show()

## US Nuclear Plants

In [ ]:
con.sql("""
    SELECT name, capacity_mw, commissioning_year, owner
    FROM plants.us_nuclear_plants
    ORDER BY capacity_mw DESC
    LIMIT 20
""").show()

## NRC Reactor Status (Recent)

In [ ]:
con.sql("""
    SELECT *
    FROM regulatory.nrc_reactor_status
    LIMIT 20
""").show()

## Power Plants Near a Location

Find plants within ~100km of a given latitude/longitude.

In [ ]:
# Example: Plants near Chicago (41.88, -87.63)
lat, lon = 41.88, -87.63

con.sql(f"""
    SELECT 
        name,
        primary_fuel,
        capacity_mw,
        ROUND(latitude, 2) as lat,
        ROUND(longitude, 2) as lon
    FROM plants.global_power_plants
    WHERE country = 'USA'
      AND ABS(latitude - {lat}) < 1.0
      AND ABS(longitude - {lon}) < 1.0
    ORDER BY capacity_mw DESC
    LIMIT 15
""").show()

## Export to Pandas (for visualization)

In [ ]:
# Convert to pandas DataFrame
df = con.sql("""
    SELECT primary_fuel, SUM(capacity_mw) as total_mw
    FROM plants.global_power_plants
    GROUP BY primary_fuel
    ORDER BY total_mw DESC
""").df()

df.head(10)

In [ ]:
# Simple bar chart
df.set_index('primary_fuel').head(10).plot(kind='barh', figsize=(10, 6), title='Global Capacity by Fuel Type (MW)')